In [3]:
from transformers import pipeline

# Set up the inference pipeline using a model from the 🤗 Hub
sentiment_analysis = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

# Let's run the sentiment analysis on each tweet
tweets = []
def sentiment_analyzer(element):
   try:
     sentiment = sentiment_analysis(element)

   except:
     pass
   return sentiment[0]['label'], sentiment


ModuleNotFoundError: No module named 'transformers'

In [2]:
import pandas as pd

df = pd.read_json('data/from_2020_06_01/abortion_clinic.json', lines = True)
df

,_type,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,...,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card
0,snscrape.modules.twitter.Tweet,https://twitter.com/DustinHapli/status/1569970...,2022-09-14 08:44:27+00:00,US: New Mexico earmarks millions for abortion ...,US: New Mexico earmarks millions for abortion ...,1569970319464570880,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,NaN,None,None,None,None,None,None,{'_type': 'snscrape.modules.twitter.SummaryCar...
1,snscrape.modules.twitter.Tweet,https://twitter.com/ML73050761/status/15699686...,2022-09-14 08:37:38+00:00,@ReadeAlexandra Of interest?\nThom Hartman her...,@ReadeAlexandra Of interest?\nThom Hartman her...,1569968605445476354,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,{'_type': 'snscrape.modules.twitter.PlayerCard...
2,snscrape.modules.twitter.Tweet,https://twitter.com/ExodusAnon178/status/15699...,2022-09-14 08:33:51+00:00,@xtinacomputes You should care. Remember how F...,@xtinacomputes You should care. Remember how F...,1569967652377956352,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.569439e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,None
3,snscrape.modules.twitter.Tweet,https://twitter.com/ML73050761/status/15699649...,2022-09-14 08:22:56+00:00,@LauraRBelin \nThom Hartman here notes that Re...,@LauraRBelin \nThom Hartman here notes that Re...,1569964903515013120,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,{'_type': 'snscrape.modules.twitter.PlayerCard...
4,snscrape.modules.twitter.Tweet,https://twitter.com/andyjlane/status/156996435...,2022-09-14 08:20:44+00:00,@mrs_coyle Has the abortion clinic thing happe...,@mrs_coyle Has the abortion clinic thing happe...,1569964350022234113,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.569785e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46486,snscrape.modules.twitter.Tweet,https://twitter.com/WentRogue/status/152220431...,2022-05-05 13:19:24+00:00,“When I was a patient advocate at an abortion ...,“When I was a patient advocate at an abortion ...,1522204313099067394,"{'_type': 'snscrape.modules.twitter.User', 'us...",4,17,128,...,NaN,None,NaN,None,None,None,None,None,None,None
46487,snscrape.modules.twitter.Tweet,https://twitter.com/shadowbear_b/status/152220...,2022-05-05 13:18:26+00:00,@thegoodgodabove there will IMMEDIATELY be an ...,@thegoodgodabove there will IMMEDIATELY be an ...,1522204072111136768,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.521996e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,None
46488,snscrape.modules.twitter.Tweet,https://twitter.com/staceysimmons/status/15222...,2022-05-05 13:17:18+00:00,@mjs_DC @ProfMMurray I worked in a Louisiana a...,@mjs_DC @ProfMMurray I worked in a Louisiana a...,1522203785061277696,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.522045e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,None,{'_type': 'snscrape.modules.twitter.SummaryCar...
46489,snscrape.modules.twitter.Tweet,https://twitter.com/cehju1111/status/152220377...,2022-05-05 13:17:16+00:00,ALSO.... going to abortion clinic was HER choi...,ALSO.... going to abortion clinic was HER choi...,1522203775963914241,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,...,N